# Day 15

## Part I

题目理解没有什么难度，关键在性能，如果按照题目描述，一路往下计算数列的下一个值，每次都回头查找最后一个数字在数列中最后一次出现的位置（或者根本不存在，为0），那么计算量及其巨大，显然无法在可接受时间内完成。

下面首先采用了一个字典，用来记录每个出现过的数字在数列中最后一次出现的位置，然后当计算下一个数字时，只需要在字典中去查找这个数字是否存在，如果存在则计算当前位置和其最后一个位置之间的差值称为下一个数字，如果不存在则为0：

In [1]:
from typing import List

def count_to_turn(serie: List[int], turn: int=2020) -> int:
    # 首先将除最后一个数字外的其他数字及其在数列中的位置加入records字典
    records = {n: i for i, n in enumerate(serie[:-1])}
    l = len(serie)
    # 当前最后一个数字即为数列最后一项
    age = serie[-1]
    while True:
        # 达到目标，直接返回最后一个数字即可
        if l == turn:
            return age
        # 如果records中不存在age，则将当前位置记录在records中的age键上，age为0
        if age not in records:
            records[age] = l - 1
            age = 0
        # 否则，计算当前位置和最后一次出现位置的差值，同样也需要记录当前位置在age键上
        else:
            last = records[age]
            records[age] = l - 1
            age = l - last - 1
        l += 1

单元测试：

In [2]:
assert(count_to_turn([0, 3, 6], 2020) == 436);
assert(count_to_turn([1, 3, 2], 2020) == 1);
assert(count_to_turn([2, 1, 3], 2020) == 10);
assert(count_to_turn([1, 2, 3], 2020) == 27);
assert(count_to_turn([2, 3, 1], 2020) == 78);
assert(count_to_turn([3, 2, 1], 2020) == 438);
assert(count_to_turn([3, 1, 2], 2020) == 1836);

第一部分结果：

In [3]:
count_to_turn([8, 11, 0, 19, 1, 2], 2020)

447

## Part II

第二部分，需要计算一个三千万的迭代，计算量相当巨大。因此我们需要重新思考策略。

下面我们使用空间换时间策略，采用一个足够大的数组（因为数列中可能出现的数不可能比迭代次数更大，因此数组长度可以直接使用迭代次数保证足够大），并且使用Numpy数组进一步提升性能，当然这样还不够，还需要使用numba的nopython jit与numpy配合，已达到进一步提升性能的目标。改进后的函数版本：

In [4]:
import numpy as np
from numba import jit, njit

@njit
def count_to_turn_array(serie: np.ndarray, turn: int=2020) -> int:
    records = np.zeros(turn, dtype=np.int32)
    for i, n in enumerate(serie[:-1]):
        records[n] = i + 1
    l = len(serie)
    age = serie[-1]
    while True:
        if l == turn:
            return age
        # 如果records中age项目为0，则将当前位置记录在records中的age键上，age为0
        if not records[age]:
            records[age] = l
            age = 0
        # 否则，计算当前位置和最后一次出现位置的差值，同样也需要记录当前位置在age键上
        else:
            last = records[age]
            records[age] = l
            age = l - last
        l += 1

单元测试：

In [6]:
assert(count_to_turn_array(np.array([0, 3, 6]), 30_000_000) == 175594);
assert(count_to_turn_array(np.array([1, 3, 2]), 30_000_000) == 2578);
assert(count_to_turn_array(np.array([2, 1, 3]), 30_000_000) == 3544142);
assert(count_to_turn_array(np.array([1, 2, 3]), 30_000_000) == 261214);
assert(count_to_turn_array(np.array([2, 3, 1]), 30_000_000) == 6895259);
assert(count_to_turn_array(np.array([3, 2, 1]), 30_000_000) == 18);
assert(count_to_turn_array(np.array([3, 1, 2]), 30_000_000) == 362);

第二部分结果：

In [8]:
count_to_turn_array(np.array([8, 11, 0, 19, 1, 2]), 30000000)

11721679

看看性能情况：

In [9]:
%timeit count_to_turn_array(np.array([8, 11, 0, 19, 1, 2]), 30000000)

525 ms ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


事实上，这样优化之后，这个运行时间比使用Rust的同样算法还要短，不得不佩服Numpy和Numba的优化效果。

第一部分同样可以采用下面这个函数进行计算：

In [10]:
count_to_turn_array([8, 11, 0, 19, 1, 2], 2020)

447